# step01. Make source text
- In task 2, text from different sections must be extracted.
- However, it is difficult to find the section in ../preprocessing/01_data4finetune/SNUH_visit_2011to2020.
- Therefore, the preprocessing started from Excel file (../sourcedata/???.xlsx) containing the sections.

In [ ]:
import preprocessing

In [ ]:
sources = ['../sources/visits_2011to2020/*.xlsx']
docperfile = 50

In [ ]:
pools = []
train_pool = "../preprocessing/01_data4finetune/pts_SNUH_visit_2011to2020_heldout_train.txt"
test_pool  = "../preprocessing/01_data4finetune/pts_SNUH_visit_2011to2020_heldout_test.txt"

def target_ptnums(target_file):
    target_pool = []
    file = open(target_file, "r")
    lines = file.readlines()
    for l in range(len(lines)):
        line = lines[l].strip("\n")
        ptnum = line.split("/")[-1].replace(".txt", "")
        target_pool.append(ptnum)
    file.close()
    
    return target_pool

pools_train = target_ptnums(train_pool)
pools_test  = target_ptnums(test_pool)
pools = pools_train + pools_test
print("len(pools): ", len(pools))
print(pools[:10])

In [ ]:
import glob
import pandas as pd
import os

for s in range(len(sources)):
    outputdir = sources[s].split("/")[-2]
    output_directory = "./data/"+str(outputdir)
    print("output_directory: ", output_directory)
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    
    filenames = glob.glob(sources[s])
    filenames.sort()
    print(filenames)
    for f in range(len(filenames)):
        reading_file = filenames[f].split("/")[-1]
        print("reading_file: ", reading_file)

        print("reading...")
        ws = pd.read_excel(filenames[f])

        print("processing...")
        
        print(str(f) + " / " + str(len(filenames)) + "..." + filenames[f])
        for row in range(len(ws)):
            if (row+1)%1000==0:
                print(str(row) + " / " + str(len(ws)))
            
            # 각 row 마다 처리
            outtext = []
            
            pt_id = ws['fake_id'][row]
#             print("pt_id: ", pt_id)
            
    
            # process target pools only
            zeros = "".join(["0"]*(8-len(str(pt_id))))
            pt_id_str = zeros + str(pt_id)
            
            if pt_id_str not in pools:
                continue
            
            date = ws['수진(진료)일'][row]
            
            doctype = ws['서식명'][row]
            doctype = preprocessing.text_preprocessing(doctype)        
            doctype = "".join(doctype)
            doctype = doctype.replace("\n", " ")
#             print("doctype: ", doctype)
            
            section = ws['서식항목명'][row]
            section = preprocessing.text_preprocessing(section)
            section = "".join(section)
            section = section.replace("\n", " ")
#             print("section: ", section)
            
            content = ws['서식내용'][row]
            content = preprocessing.text_preprocessing(content)
#             print("content: ", content)
            
            
            for c in range(len(content)):
                content_sub = content[c].split("\n")
                for s in range(len(content_sub)):
                    if content_sub[s]!="":
                        outtext.append(date+"\t"+doctype+"\t"+section+"\t"+content_sub[s])
            
            target_folder = output_directory+"/"+str(((pt_id//docperfile)+1)*docperfile)
            if not os.path.exists(target_folder):
                os.makedirs(target_folder)
                
            file = open(target_folder+"/"+pt_id_str+".txt", "a")
            file.write("\n".join(outtext)+"\n")
            file.close()
            

# step02. Dictionary of 'section to patient id'
- make a dictionary which cover section name to filename(=patient fake id)

In [ ]:
import glob
import os

paths = [
    "./data/visits_2011to2020/*"
]

for p in range(len(paths)):
    type_to_file = {}
    
    path = paths[p]
    groups = glob.glob(path)
    groups.sort()
    
    category = path.split("/")[-2]
    
    for g in range(len(groups)):
        files = glob.glob(groups[g]+"/*.txt")
        files.sort()
        
        print(str(g)+"/"+str(len(groups)) +" ... "+str(path))
#         print("groups[g]: ", groups[g])
        
        for f in range(len(files)):
            file = open(files[f], "r")
            lines = file.readlines()
#             print("files[f]: ", files[f])
            
            #print(str(f)+"/"+str(len(files))+" ... "+str(g)+"/"+str(len(groups)) +"..."+str(path))
            
            for l in range(len(lines)):
                line = lines[l]
#                 print("line: ", line)
                if line=="\n":
                    continue
                date = line.split("\t")[0]
                doctype = line.split("\t")[1]
                section = line.split("\t")[2]
                content_type = doctype+"\t"+section
#                 print("content_type: ", content_type)
                
                # type 과 파일 링크
                if content_type not in type_to_file:
                    type_to_file[content_type] = [files[f]]
                else:
                    link_paths = type_to_file[content_type]
                    if files[f] not in link_paths:
                        if len(link_paths)<=10:
                            link_paths.append(files[f])
                        
                    type_to_file[content_type] = link_paths
        
    
    outtext = []
    for key in type_to_file:
        vals = type_to_file[key]
        for v in range(len(vals)):
            outtext.append(key+"\t"+vals[v])
    
    target_folder = "./data/02_type_to_file_links"
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    file = open(target_folder+"/sections_"+str(category)+"_task2.txt", "w")
    file.write("\n".join(outtext))
    file.close()


# # Sections_visits_2011to2020_task2.txt file was manually curated to classify what was assessment & plan and what was not.